In [1]:
import time
import random
import pickle
import urllib
import logging
from IPython.display import Image
from IPython.core.display import HTML 
from requests import HTTPError
import tmdbsimple as tmdb

In [2]:
tmdb.API_KEY = '0a6964b42f986ab0794414f5df403d7e'
pickle_file_name = 'data/tmdb_info.pickle3'

In [3]:
logger = logging.getLogger('tmdb')
log_file = logging.FileHandler('data/tmdb.log')
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
log_file.setFormatter(formatter)
logger.addHandler(log_file) 
logger.setLevel(logging.WARNING)

In [4]:
# initialize master list
try:
    with open(pickle_file_name, 'rb') as handle:
        master_dict = pickle.load(handle)
except IOError:
        master_dict = {}
initial_master_size = len(master_dict)
print 'Initital master size:' + str(initial_master_size)

Initital master size:12244


In [5]:
conf = tmdb.Configuration()
c_info = conf.info()
img_conf = c_info['images']

In [6]:
# latest movie id, out upper bound for random movie id
latest_movie = tmdb.Movies().latest()
latest_id = latest_movie['id']
start_id = 1
end_id = latest_id

In [7]:
# load batch
batch_size = 4000
loaded_count = 0
while(loaded_count < batch_size):
    # try to get movie so many times
    limit = 100
    attempts = 0
    while(attempts < limit):
        # generate random movie id
        random_id = random.randrange(start_id, end_id)
        attempts += 1
        if(random_id in master_dict):
            continue # already have this movie
        random_movie = tmdb.Movies(random_id)
        try:
            time.sleep(1)
            info = random_movie.info()
            if(len(random_movie.genres) == 0):
                continue # no genres, skip
            if(random_movie.poster_path is None):
                continue # no poster, skip
            # save poster to a file
            image_url = img_conf['base_url'] + 'w500' + random_movie.poster_path
            urllib.urlretrieve(image_url, 'posters/' + str(random_id) + '.jpg')
            # poster saved, get credits
            time.sleep(1)
            credits = random_movie.credits()
            break
        except HTTPError as e:
            if e.response.status_code != 404:
                logger.error('Failed to get movie for id: ' + str(random_movie) + ', ' + repr(e))
            time.sleep(1)
        except IOError as e:
            logger.error('Failed to get movie for id: ' + str(random_movie) + ', ' + repr(e))
            time.sleep(1)
    # we failed to get movie
    if(attempts == limit):
        logger.error('Attempted: ' + str(attempts) + ' times, exiting...')
        break
    # add info to the master list
    loaded_count += 1
    master_dict[random_id] = random_movie

In [8]:
log_file.close()

In [9]:
if(len(master_dict) > initial_master_size):
    print('Saved ' + str(len(master_dict)) + ' movies.')
    with open(pickle_file_name, 'wb') as handle:
        pickle.dump(master_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

Saved 16244 movies.
